## ASHRAE 140 2023

Section 6: Weather Test

In [1]:
# Add to environment variables PYTHONPATH = C:\Users\JFC_WIN\Documents\OpenSimula\src
import OpenSimula as osm
import pandas as pd

case_WD = {
    "name": "case_WD",
    "time_step": 3600,
    "n_time_steps": 8760,
    "initial_time": "01/01/2001 00:00:00",
    "components": [
        # MET_FILE
        {
            "type": "File_met",
            "name": "TMY3",
            "file_type": "TMY3",
            "file_name": "WD100.tmy3"
        },
        # CONSTRUCTION
        {
            "type": "Material",
            "name": "Madera",
            "conductivity": 0.14,
            "density": 530,
            "specific_heat": 900
        },
        {
            "type": "Construction",
            "name": "Pared",
            "solar_alpha": [1, 1],
            "materials": ["Madera"],
            "thicknesses": [0.10],
        },
        # BUILDING
        {
            "type": "Building",
            "name": "WT_Building",
            "file_met": "TMY3",
            "albedo": 0.0
        },
        {
            "type": "Space_type",
            "name": "ashrae_space",
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "4.1667",
            "other_gains_radiant_fraction": 0.6,
            "infiltration": "1"
        },
        {
            "type": "Space",
            "name": "space",
            "building": "WT_Building",
            "space_type": "ashrae_space",
            "floor_area": 48,
            "volume": 48*2.7,
        },
        # Surfaces
        {
            "type": "Exterior_surface",
            "name": "north_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [8,6,0],
            "width": 8,
            "height": 2.7,
            "azimuth": 180,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "west_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [0,6,0],
            "width": 4,
            "height": 2.7,
            "azimuth": -90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_west_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [0,2,0],
            "width": 2.828,
            "height": 2.7,
            "azimuth": -45,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [2,0,0],
            "width": 4,
            "height": 2.7,
            "azimuth": 0,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_east_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [6,0,0],
            "width": 2.828,
            "height": 2.7,
            "azimuth": 45,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "east_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [8,2,0],
            "width": 4,
            "height": 2.7,
            "azimuth": 90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [2,2,3.85],
            "width": 4,
            "height": 4,
            "azimuth": 0,
            "altitude": 90
        },
        {
            "type": "Exterior_surface",
            "name": "west_roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [0,6,2.7],
            "width": 4,
            "height": 2.307,
            "azimuth": -90,
            "altitude": 60
        },
        {
            "type": "Exterior_surface",
            "name": "east_roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [8,2,2.7],
            "width": 4,
            "height": 2.307,
            "azimuth": 90,
            "altitude": 60
        },
        {
            "type": "Exterior_surface",
            "name": "south_roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [2,0,2.7],
            "width": 4,
            "height": 2.307,
            "azimuth": 0,
            "altitude": 60
        },
    ],
}

In [2]:
sim = osm.Simulation()
pro = osm.Project("pro", sim)
pro.read_dict(case_WD)

Reading project data from dictonary
Reading completed.
Checking project: case_WD
ok


In [3]:
pro.component("WT_Building").draw_pyvista()

Widget(value='<iframe src="http://localhost:55627/index.html?ui=P_0x1114c89d0_0&reconnect=auto" class="pyvista…

In [4]:
pro.simulate()

Simulating case_WD: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


In [5]:
df_met = pro.component("TMY3").variable_dataframe()
df_graph = pd.DataFrame()
df_graph["date"] = df_met["date"]
df_graph["T"] = df_met["temperature [°C]"]
df_graph["RH"] = df_met["rel_humidity [%]"]
df_graph["T_DP"] = df_met["dew_point_temp [°C]"]
df_graph["W"] = df_met["abs_humidity [g/kg]"]/1000
df_graph["T_WB"] = df_met["wet_bulb_temp [°C]"]
df_graph["wind_speed"] = df_met["wind_speed [m/s]"]
df_graph["wind_direction"] = df_met["wind_direction [°]"]
df_graph["p"] = df_met["pressure [Pa]"]/100
df_graph["total_cloud_cover"] = df_met["total_cloud_cover [%]"]/10
df_graph["opaque_cloud_cover"] = df_met["opaque_cloud_cover [%]"]/10
df_graph["T_sky"] = df_met["sky_temperature [°C]"]
df_graph["h_solar"] = df_met["sol_hour [h]"]
df_graph["hor_tot"] = pro.component("roof").variable("E_dir0").values +  pro.component("roof").variable("E_dif0").values
df_graph["hor_beam"] = pro.component("roof").variable("E_dir0").values 
df_graph["hor_dif"] = pro.component("roof").variable("E_dif0").values
df_graph["south_tot"] = pro.component("south_wall").variable("E_dir0").values +  pro.component("south_wall").variable("E_dif0").values
df_graph["south_beam"] = pro.component("south_wall").variable("E_dir0").values 
df_graph["south_dif"] = pro.component("south_wall").variable("E_dif0").values
df_graph["east_tot"] = pro.component("east_wall").variable("E_dir0").values +  pro.component("east_wall").variable("E_dif0").values
df_graph["east_beam"] = pro.component("east_wall").variable("E_dir0").values 
df_graph["east_dif"] = pro.component("east_wall").variable("E_dif0").values
df_graph["north_tot"] = pro.component("north_wall").variable("E_dir0").values +  pro.component("north_wall").variable("E_dif0").values
df_graph["north_beam"] = pro.component("north_wall").variable("E_dir0").values 
df_graph["north_dif"] = pro.component("north_wall").variable("E_dif0").values
df_graph["west_tot"] = pro.component("west_wall").variable("E_dir0").values +  pro.component("west_wall").variable("E_dif0").values
df_graph["west_beam"] = pro.component("west_wall").variable("E_dir0").values 
df_graph["west_dif"] = pro.component("west_wall").variable("E_dif0").values
df_graph["south_east_tot"] = pro.component("south_east_wall").variable("E_dir0").values +  pro.component("south_east_wall").variable("E_dif0").values
df_graph["south_east_beam"] = pro.component("south_east_wall").variable("E_dir0").values 
df_graph["south_east_dif"] = pro.component("south_east_wall").variable("E_dif0").values
df_graph["south_west_tot"] = pro.component("south_west_wall").variable("E_dir0").values +  pro.component("south_west_wall").variable("E_dif0").values
df_graph["south_west_beam"] = pro.component("south_west_wall").variable("E_dir0").values 
df_graph["south_west_dif"] = pro.component("south_west_wall").variable("E_dif0").values
df_graph["east_roof_tot"] = pro.component("east_roof").variable("E_dir0").values +  pro.component("east_roof").variable("E_dif0").values
df_graph["east_roof_beam"] = pro.component("east_roof").variable("E_dir0").values 
df_graph["east_roof_dif"] = pro.component("east_roof").variable("E_dif0").values
df_graph["south_roof_tot"] = pro.component("south_roof").variable("E_dir0").values +  pro.component("south_roof").variable("E_dif0").values
df_graph["south_roof_beam"] = pro.component("south_roof").variable("E_dir0").values 
df_graph["south_roof_dif"] = pro.component("south_roof").variable("E_dif0").values
df_graph["west_roof_tot"] = pro.component("west_roof").variable("E_dir0").values +  pro.component("west_roof").variable("E_dif0").values
df_graph["west_roof_beam"] = pro.component("west_roof").variable("E_dir0").values 
df_graph["west_roof_dif"] = pro.component("west_roof").variable("E_dif0").values



df_graph.to_excel("WD100_OpenSimula.xlsx")
df_met.head(24)

,date,sol_hour [h],temperature [°C],sky_temperature [°C],underground_temperature [°C],rel_humidity [%],abs_humidity [g/kg],dew_point_temp [°C],wet_bulb_temp [°C],sol_direct [W/m²],sol_diffuse [W/m²],wind_speed [m/s],wind_direction [°],sol_azimuth [°],sol_altitude [°],pressure [Pa],total_cloud_cover [%],opaque_cloud_cover [%]
0,2001-01-01 00:30:00,0.474926,-18.0,-35.364197,10.875342,85.0,0.790008,-19.708862,-18.315822,0.0,0.0,0.0,0.0,0.000000,0.000000,83700.0,20.0,20.0
1,2001-01-01 01:30:00,1.474926,-16.6,-33.191892,10.875342,83.0,0.882094,-18.578841,-16.999537,0.0,0.0,0.0,0.0,0.000000,0.000000,83500.0,30.0,30.0
2,2001-01-01 02:30:00,2.474926,-15.3,-31.599713,10.875342,85.0,1.020834,-17.045252,-15.688165,0.0,0.0,0.0,0.0,0.000000,0.000000,83400.0,30.0,30.0
3,2001-01-01 03:30:00,3.474926,-14.4,-29.911318,10.875342,90.0,1.178902,-15.542106,-14.676325,0.0,0.0,0.0,0.0,0.000000,0.000000,83100.0,40.0,40.0
4,2001-01-01 04:30:00,4.474926,-13.0,-27.914910,10.875342,86.0,1.275778,-14.649535,-13.425674,0.0,0.0,0.0,0.0,0.000000,0.000000,83400.0,50.0,50.0
5,2001-01-01 05:30:00,5.474926,-11.1,-25.060021,10.875342,88.0,1.557626,-12.520039,-11.515680,0.0,0.0,0.0,0.0,0.000000,0.000000,83000.0,60.0,60.0
6,2001-01-01 06:30:00,6.474926,-9.7,-23.524533,10.875342,84.0,1.686621,-11.653733,-10.307203,0.0,0.0,0.0,0.0,0.000000,0.000000,82900.0,60.0,60.0
7,2001-01-01 07:30:00,7.474926,-7.7,-20.487081,10.875342,82.0,1.966177,-9.955406,-8.473369,0.0,9.0,1.4,120.0,58.513402,0.904630,82800.0,70.0,70.0
8,2001-01-01 08:30:00,8.474926,-5.9,-18.413814,10.875342,80.0,2.240256,-8.467897,-6.853244,12.0,77.0,1.3,140.0,48.206136,10.147571,82900.0,70.0,70.0
9,2001-01-01 09:30:00,9.474926,-2.2,-16.349949,10.875342,66.0,2.537383,-7.074146,-4.200054,151.0,115.0,1.7,200.0,36.432616,17.909163,82700.0,80.0,40.0


In [6]:
df_eplus = pd.read_excel("WeatherDriversResults_EnergyPlus.xlsx", sheet_name="WD100", header=None, skiprows=2)
df_eplus["time"] = pd.date_range(start ='01-01-2001 00:00:00', periods = 8760*6, freq ='10min')
df_eplus

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,time
0,0.083333,-7.166667,55.833333,-13.950000,0.001411,-9.167844,7.333333,8.333333,822.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01 00:00:00
1,0.250000,-9.333333,61.666667,-15.100000,0.001284,-10.845223,5.866667,6.666667,825.0,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01 00:10:00
2,0.416667,-11.500000,67.500000,-16.250000,0.001155,-12.608003,4.400000,5.000000,828.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01 00:20:00
3,0.583333,-13.666667,73.333333,-17.400000,0.001027,-14.446757,2.933333,3.333333,831.0,1.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01 00:30:00
4,0.750000,-15.833333,79.166667,-18.550000,0.000905,-16.352318,1.466667,1.666667,834.0,1.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01 00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,8759.250000,-18.666667,83.333333,-20.566667,0.000724,-18.998463,1.400000,333.333333,841.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-12-31 23:10:00
52556,8759.416667,-18.850000,82.500000,-20.850000,0.000704,-19.193417,1.050000,340.000000,841.0,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-12-31 23:20:00
52557,8759.583333,-19.033333,81.666667,-21.133333,0.000685,-19.387948,0.700000,346.666667,841.0,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-12-31 23:30:00
52558,8759.750000,-19.216667,80.833333,-21.416667,0.000666,-19.582065,0.350000,353.333333,841.0,0.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-12-31 23:40:00


In [12]:
import plotly.express as px

#fig = px.line(df_graph,x='date',y=['T_ext','T_z','T_s0','T_s1'],height=400)
fig = px.line(df_graph,x='date',y=['east_dif','south_dif'],height=400)
#fig = px.line(df_graph,x='date',y='east_dif',height=400)
#fig.update_traces(line_color="red")
#fig2 = px.line(df_eplus,x='time',y=20)
#fig.add_trace(fig2.data[0])

fig.show()